In [2]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
from datetime import datetime

from __future__ import print_function
import pickle

In [3]:
url = 'https://www.the-numbers.com/movie/budgets/all'

user_agent = {'User-agent': 'Mozilla/5.0'}

response  = requests.get(url, headers = user_agent)

page = response.text

soup = BeautifulSoup(page,"lxml")

In [4]:
table = soup.find('table')
table

<table>
<tr><th> </th><th>Release<br/>Date</th><th>Movie</th><th>Production<br/>Budget</th><th>Domestic<br/>Gross</th><th>Worldwide<br/>Gross</th></tr>
<tr><td class="data">1</td>
<td><a href="/box-office-chart/daily/2019/04/23">Apr 23, 2019</a></td>
<td><b><a href="/movie/Avengers-Endgame-(2019)#tab=summary">Avengers: Endgame</a></b></td>
<td class="data"> $400,000,000</td>
<td class="data"> $858,373,000</td>
<td class="data"> $2,797,800,564</td>
</tr>
<tr><td class="data">2</td>
<td><a href="/box-office-chart/daily/2011/05/20">May 20, 2011</a></td>
<td><b><a href="/movie/Pirates-of-the-Caribbean-On-Stranger-Tides#tab=summary">Pirates of the Caribbean: On Stranger Tides</a></b></td>
<td class="data"> $379,000,000</td>
<td class="data"> $241,063,875</td>
<td class="data"> $1,045,663,875</td>
</tr>
<tr><td class="data">3</td>
<td><a href="/box-office-chart/daily/2015/04/22">Apr 22, 2015</a></td>
<td><b><a href="/movie/Avengers-Age-of-Ultron#tab=summary">Avengers: Age of Ultron</a></b></

In [5]:
rows = [row for row in table.find_all('tr')]

In [6]:
movies = {}

for row in rows[1:]:
    items = row.find_all('td')
    link = row.find('a')
    title, url = link.text, link['href']
    movies[title] = [url] + [i.text for i in items]
    
movies

{'Apr 23, 2019': ['/box-office-chart/daily/2019/04/23',
  '1',
  'Apr 23, 2019',
  'Avengers: Endgame',
  '\xa0$400,000,000',
  '\xa0$858,373,000',
  '\xa0$2,797,800,564'],
 'May 20, 2011': ['/box-office-chart/daily/2011/05/20',
  '2',
  'May 20, 2011',
  'Pirates of the Caribbean: On Stranger Tides',
  '\xa0$379,000,000',
  '\xa0$241,063,875',
  '\xa0$1,045,663,875'],
 'Apr 22, 2015': ['/box-office-chart/daily/2015/04/22',
  '3',
  'Apr 22, 2015',
  'Avengers: Age of Ultron',
  '\xa0$365,000,000',
  '\xa0$459,005,868',
  '\xa0$1,396,099,202'],
 'Dec 16, 2015': ['/box-office-chart/daily/2015/12/16',
  '4',
  'Dec 16, 2015',
  'Star Wars Ep. VII: The Force Awakens',
  '\xa0$306,000,000',
  '\xa0$936,662,225',
  '\xa0$2,065,478,084'],
 'Apr 25, 2018': ['/box-office-chart/daily/2018/04/25',
  '5',
  'Apr 25, 2018',
  'Avengers: Infinity War',
  '\xa0$300,000,000',
  '\xa0$678,815,482',
  '\xa0$2,044,540,523'],
 'May 24, 2007': ['/box-office-chart/daily/2007/05/24',
  '6',
  'May 24, 2007'

In [15]:
budget1 = pd.DataFrame(movies).T  #transpose
budget1.columns = ['link_stub','rank_all_movies','release','title', 
                    'budget', 'domestic_gross','worldwide_gross']

budget1.head()

,link_stub,rank_all_movies,release,title,budget,domestic_gross,worldwide_gross
"Apr 23, 2019",/box-office-chart/daily/2019/04/23,1,"Apr 23, 2019",Avengers: Endgame,"$400,000,000","$858,373,000","$2,797,800,564"
"May 20, 2011",/box-office-chart/daily/2011/05/20,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$379,000,000","$241,063,875","$1,045,663,875"
"Apr 22, 2015",/box-office-chart/daily/2015/04/22,3,"Apr 22, 2015",Avengers: Age of Ultron,"$365,000,000","$459,005,868","$1,396,099,202"
"Dec 16, 2015",/box-office-chart/daily/2015/12/16,4,"Dec 16, 2015",Star Wars Ep. VII: The Force Awakens,"$306,000,000","$936,662,225","$2,065,478,084"
"Apr 25, 2018",/box-office-chart/daily/2018/04/25,5,"Apr 25, 2018",Avengers: Infinity War,"$300,000,000","$678,815,482","$2,044,540,523"


In [26]:
budget1['budget'] = budget1['budget'].str.replace('$', '')
budget1['budget'] = budget1['budget'].str.replace(',', '')
budget1['budget'] = budget1['budget'].astype(int)

In [29]:
budget1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 100 entries, Apr 23, 2019 to Jul 20, 2017
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   link_stub        100 non-null    object
 1   rank_all_movies  100 non-null    object
 2   release          100 non-null    object
 3   title            100 non-null    object
 4   budget           100 non-null    int64 
 5   domestic_gross   100 non-null    object
 6   worldwide_gross  100 non-null    object
dtypes: int64(1), object(6)
memory usage: 6.2+ KB


In [30]:
budget1.to_csv('budget1.csv')